In [8]:
import voila
import numpy as np
import matplotlib as mpl
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
import json
import base64

# Armchair Graphene Nanoribbon Junctions Explorer

Select the parameters of the junction you want to investigate. For every junction, you can download files containing all the informations about it by clicking on the button at the bottom of the page. If you use this tool, please cite the following works:

**J. Leuenberger, K. Cernevics, O. V. Yazyev, Optimizing Graphene-Based Nanoelectronics via Junction Edge Characterization**


**J. Leuenberger, K. Cernevics, O. V. Yazyev, Armchair Graphene Nanoribbon Junctions Explorer**

In [9]:
def get_r_max(N_value):
    security = 1
    return np.floor(N_value / 2) - 1 - security

In [10]:
shape = widgets.Dropdown(options=['armchair', 'round', 'zigzag'], value='armchair', description='Shape')
N = widgets.IntSlider(min=20, max=80, step=3, description="N")
wext = widgets.IntSlider(min=0, max=get_r_max(N.value), step=1, description="wext")
a_ratio = widgets.FloatText(value=-1, description='A. ratio', disabled=True)
z_ratio = widgets.FloatText(value=-1, description='Z. ratio', disabled=True)
tau_around_0p00_ev = widgets.FloatText(value=-1, description='τ @ 0.00 eV', disabled=True)
tau_around_0p01_ev = widgets.FloatText(value=-1, description='τ @ 0.01 eV', disabled=True)
tau_around_0p10_ev = widgets.FloatText(value=-1, description='τ @ 0.10 eV', disabled=True)

html_button_resume = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button">-</button>
</a>
</body>
</html>
""")

html_button_conductance = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button">-</button>
</a>
</body>
</html>
""")

html_button_vertices = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button">-</button>
</a>
</body>
</html>
""")


html_button_edges = widgets.HTML(value="""<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="" href="data:text/json;base64," download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button">-</button>
</a>
</body>
</html>
""")

In [11]:
def update_image(shape, N, r):
    fig, axs = plt.subplots(ncols=4, nrows=1, figsize=(10, 5))
    try:
        axs[0].imshow(mpl.image.imread(f"./data/shape={shape}/N={N}/wext={r}/edge.png"))
        axs[1].imshow(mpl.image.imread(f"./data/shape={shape}/N={N}/wext={r}/conductance.png"))
        axs[2].imshow(mpl.image.imread(f"./data/shape={shape}/N={N}/wext={r}/ldos.png"))
        axs[3].imshow(mpl.image.imread(f"./data/shape={shape}/N={N}/wext={r}/current.png"))
        for ax in axs.ravel():
            ax.axis("off")
        fig.tight_layout()
    except Exception as e:
        # print(e)
        # plt.close()
        # print(f"Missing data !")
        pass
    
def update_r_max(*args):
    r_current = wext.value
    r_max = get_r_max(N.value)
    if r_current > r_max:
        wext.value = r_max
    wext.max = r_max

def update_texts(change):
    try:
        infos = json.load(open(f"./data/shape={shape.value}/N={N.value}/wext={wext.value}/summary.json"))
        a_ratio.value = infos["armchair_ratio"]
        z_ratio.value = infos["zigzag_ratio"]
        tau_around_0p00_ev.value = infos["tau_around_0p00_ev"]
        tau_around_0p01_ev.value = infos["tau_around_0p01_ev"]
        tau_around_0p10_ev.value = infos["tau_around_0p10_ev"]
    except Exception as e:
        #print(e)
        pass

def update_downloadbutton(*args):
    
    base_html = """<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="FILENAME" href="data:text/json;base64,PAYLOAD" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button">TEXT</button>
    </a>
    </body>
    </html>"""

    res = open(f"./data/shape={shape.value}/N={N.value}/wext={wext.value}/summary.json").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"summary_shape={shape.value}_N={N.value}_wext={wext.value}.json"
    html_button_resume.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Summary") # ({sys.getsizeof(payload)/1024:.1f} ko)")

    res = open(f"./data/shape={shape.value}/N={N.value}/wext={wext.value}/conductance.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"conductance_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_conductance.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Conductance") # ({sys.getsizeof(payload)/1024:.1f} ko)")

    res = open(f"./data/shape={shape.value}/N={N.value}/wext={wext.value}/vertices.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"vertices_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_vertices.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Vertices") # ({sys.getsizeof(payload)/1024:.1f} ko)")

    res = open(f"./data/shape={shape.value}/N={N.value}/wext={wext.value}/edges.csv").read()
    b64 = base64.b64encode(res.encode())
    payload = b64.decode().replace("\"", "`")
    filename = f"edges_shape={shape.value}_N={N.value}_wext={wext.value}.csv"
    html_button_edges.value = base_html.replace("FILENAME", filename).replace("PAYLOAD", payload).replace("TEXT", f"Edges") # ({sys.getsizeof(payload)/1024:.1f} ko)")


## Inputs

Here you can select the parameters of the junction you want to investigate. Note that the maximum amount of roundening $w_\mathrm{ext}$ does depend on the value of $N$.

In [12]:
shape.observe(update_texts, names='value')
N.observe(update_texts, names='value')
wext.observe(update_texts, names='value')

N.observe(update_r_max, names='value')

shape.observe(update_downloadbutton, names='value')
N.observe(update_downloadbutton, names='value')
wext.observe(update_downloadbutton, names='value')

display(widgets.interactive(update_image, shape=shape, N=N, r=wext))

interactive(children=(Dropdown(description='Shape', options=('armchair', 'round', 'zigzag'), value='armchair')…

## Outputs

Here you can retrieve some informations about the junction. Note that all data displayed here can be downloaded as well below.

In [13]:
display(
    widgets.VBox([
        a_ratio,
        z_ratio,
        tau_around_0p00_ev,
        tau_around_0p01_ev,
        tau_around_0p10_ev,
    ]),
)

## Download

Here you can download informations about the junction.
* **Summary** contains informations about the junction's geometry and its preserved conductance.
* **Conductance** contains the conductance [$G_0$] of the junction at different energies [$\mathrm{eV}$].
* **Vertices** contains a list of the junction's vertices' positions [$a$], as well as the local density of states (LDOS) at each of them.
* **Edges** contains a list of the junction's edges, as well as the probability currents flowing at each of them.

In [14]:
display(
    widgets.HBox([
    html_button_resume,
    html_button_conductance,
    html_button_vertices,
    html_button_edges,
    ]))